In [14]:
noise = ['balanced', 'bias', 'flip']
level = ['0.1','0.3']
dataset = ['income', 'compas', 'adult', 'synthetic_20']
shifts = [ '0.5_1.5','0.25_1.1','1_2']

In [15]:
import os
import shutil
import pandas as pd
import numpy as np

In [4]:
# transforming files to drop extra column and ensuring everything is in int 
base_path = '/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/data/covariate_shifted_data/semi_synthetic'

for sh in shifts:
    for n in noise:
        for l in level:
            for ds in dataset:
                # debug code for the rest of it. 
                if ds in ['income']: continue 
                
                for f in ['train_ids.csv', 'test_ids.csv', 'valid_ids.csv']:
                    if f not in ['train_ids.csv']: continue
                    try:
                        folds = [i for i in range(1,11)]
                        df = pd.read_csv(os.path.join(base_path, sh, n, l, ds, f), usecols = [f'x{f}' for f in folds], index_col=False)
                        # print(len(df))
                        df.dropna(inplace=True)
                        # print(len(df))
                        for col in df.columns: df[col] = df[col].astype(np.int64)
                        df.to_csv(os.path.join(base_path, sh, n, l, ds, f), index=False)
                    except Exception as e:
                        print(e)
                    

                    

In [ ]:
# copying files to the right expected structure 
# works for semi synthetic folder 
base_path = '/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/data/covariate_shifted_data/semi_synthetic'
for sh in shifts:
    for n in noise:
        for l in level:
            for ds in dataset:
                
                # for f in ['train_ids.csv', 'test_ids.csv', 'valid_ids.csv']:
                try:
                    if ds in ['synthetic_20']:  new_path = os.path.join(base_path, sh, n, l, 'synthetic', '20')
                    else:   new_path = os.path.join(base_path, sh, n, l, ds)

                    old_path = os.path.join(base_path, sh, ds, n, l)
                    if os.path.exists(new_path):
                        shutil.rmtree(new_path)
                    if os.path.exists(old_path):
                        shutil.copytree(old_path, new_path)
                    else:
                        print(f'{old_path=} does not exist')
                except FileExistsError:
                    continue
                except FileNotFoundError as fe:
                    print(f'{fe=} , {old_path=}')
                    continue


In [17]:
base_path = '/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/data/covariate_shifted_data/real'
for ds in ['employment', 'income']:
    for sh in ['1', '2', '3']:
        old_path = os.path.join(base_path, ds, sh)
        new_path = os.path.join(base_path, sh, ds)
        if os.path.exists(new_path):
                        shutil.rmtree(new_path)
        if os.path.exists(old_path):
            shutil.copytree(old_path, new_path)
        else:
            print(f'{old_path=} does not exist')

In [13]:
# construct julia commands

local_path = f'/home/sanchlia/Debias'
noise = ['balanced']
level = ['0.1','0.3']
dataset = ['income']
shifts = ['0.5_1.5', '0.25_1.1','1_2']
folds = [i for i in range(1,11)]

s_map = {
    "income": 'F9'
}
for ds in dataset:
    
    # base_cmd.append(ds_cmd)
    for sh in shifts:
        for n in noise:
            for l in level:
                for f in folds:
                    f = str(f)
                    base_cmd = ['julia  --project=Project.toml /home/sanchlia/Debias/bin/learn.jl']
                    ds_cmd = f'{ds} --struct_type FairPC --sensitive_variable \"{s_map[ds]}\"'
                    base_cmd.append(ds_cmd)
                    exp_id = f'--exp-id {f}'
                    exp_path = os.path.join('/home/sanchlia/Debias/exp', ds, sh, n,l,f)
                    dir_path = f'--dir {exp_path}'
                    fold_arg = f'--fold {f}'
                    bpath = os.path.join(local_path, "noisy_datasets", ds, n, l)
                    idpath = os.path.join(local_path, 'data', 'covariate_shifted_data', 'semi_synthetic', sh, n, l)
                    base_arg = f'--base_path {bpath}'
                    id_arg = f'--id_dir {idpath}'
                    base_cmd.append(exp_id)
                    base_cmd.append(dir_path)
                    base_cmd.append(fold_arg)
                    base_cmd.append(base_arg)
                    base_cmd.append(id_arg)
                    base_cmd.append("--num_X 20")
                    base_cmd.append('&')
                    print(" ".join(base_cmd))
                    print('\n')
# julia_rosetta --project=Project.toml bin/learn.jl income --exp-id 1 
# --dir exp/income/0.5_1.5/balanced/0.1/1 
# --struct_type FairPC --sensitive_variable "F9" 
# --fold 1 
# --base_path "/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/noisy_datasets/income/balanced/0.1/" 
# --id_dir "/Users/sauravanchlia/Fair_ML/Projects/debias_shift/Debias/data/covariate_shifted_data/semi_synthetic/0.5_1.5/balanced/0.1/" &
# julia_cmd = ""

julia  --project=Project.toml /home/sanchlia/Debias/bin/learn.jl income --struct_type FairPC --sensitive_variable "F9" --exp-id 1 --dir /home/sanchlia/Debias/exp/income/0.5_1.5/balanced/0.1/1 --fold 1 --base_path /home/sanchlia/Debias/noisy_datasets/income/balanced/0.1 --id_dir /home/sanchlia/Debias/data/covariate_shifted_data/semi_synthetic/0.5_1.5/balanced/0.1 --num_X 20 &


julia  --project=Project.toml /home/sanchlia/Debias/bin/learn.jl income --struct_type FairPC --sensitive_variable "F9" --exp-id 2 --dir /home/sanchlia/Debias/exp/income/0.5_1.5/balanced/0.1/2 --fold 2 --base_path /home/sanchlia/Debias/noisy_datasets/income/balanced/0.1 --id_dir /home/sanchlia/Debias/data/covariate_shifted_data/semi_synthetic/0.5_1.5/balanced/0.1 --num_X 20 &


julia  --project=Project.toml /home/sanchlia/Debias/bin/learn.jl income --struct_type FairPC --sensitive_variable "F9" --exp-id 3 --dir /home/sanchlia/Debias/exp/income/0.5_1.5/balanced/0.1/3 --fold 3 --base_path /home/sanchlia/Debias/noisy

In [ ]:
m